<div style="text-align: right"> Tommy Evans-Barton </div>
<div style="text-align: right"> GM Draft Analysis </div>

# Data Ingestion and Cleaning

In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
import urllib
import re
import sys
from glob import glob

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TOP_PATH = os.environ['PWD']

'/Users/tevansbarton/aaasideprojects/gmdrafttendencies'

In [4]:
sys.path.append(TOP_PATH + '/config')
sys.path.append(TOP_PATH + '/src')
sys.path.append(TOP_PATH + '/src/data')
sys.path.append(TOP_PATH + '/src/processing')

In [5]:
import etl
import processing

In [19]:
with open(TOP_PATH + '/config/data-params.json') as fh:
    data_cfg = json.load(fh)

In [31]:
etl.get_data(**data_cfg)

In [32]:
draft_2018 = pd.read_csv(TOP_PATH + '/data/raw/DRAFT_2018.csv')

In [33]:
draft_2018.columns

Index(['Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To', 'AP1', 'PB', 'St',
       'G', 'Passing Cmp', 'Passing Att', 'Passing Yds', 'Passing TD',
       'Passing Int', 'Rushing Att', 'Rushing Yds', 'Rushing TD',
       'Receiving Rec', 'Receiving Yds', 'Receiving TD', 'Solo', 'Int', 'Sk',
       'College/Univ', 'YEAR'],
      dtype='object')

In [34]:
draft_2018.T

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
Rnd,1,1,1,1,1,1,1,1,1,1,...,7,7,7,7,7,7,7,7,7,7
Pick,1,2,3,4,5,6,7,8,9,10,...,247,248,249,250,251,252,253,254,255,256
Tm,CLE,NYG,NYJ,CLE,DEN,IND,BUF,CHI,SFO,ARI,...,JAX,GNB,CIN,NWE,LAC,CIN,CIN,ARI,BUF,WAS
Player,Baker Mayfield,Saquon Barkley,Sam Darnold,Denzel Ward,Bradley Chubb,Quenton Nelson,Josh Allen,Roquan Smith,Mike McGlinchey,Josh Rosen,...,Logan Cooke,Kendall Donnerson,Logan Woodside,Ryan Izzo,Justin Jackson,Rod Taylor,Auden Tate,Korey Cunningham,Austin Proehl,Trey Quinn
Pos,QB,RB,QB,CB,DE,G,QB,ILB,T,QB,...,P,OLB,QB,TE,RB,G,WR,T,WR,WR
Age,23,21,21,21,22,22,22,21,24,21,...,23,22,23,22,23,23,21,23,22,22
To,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,...,2019,NaN,NaN,2019,2019,NaN,2019,2019,NaN,2019
AP1,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PB,0,1,0,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
St,2,2,2,2,1,2,2,2,2,1,...,0,0,0,0,0,0,1,0,0,0


In [9]:
executives = pd.read_csv(TOP_PATH + '/data/raw/EXECUTIVES.csv')

In [86]:
processing.combine_drafts_and_executives()

,GM,YEAR,Rnd,Pick,Tm,Player,Pos,Age,Played To,AP1,...,Sk,College/Univ,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,Les Snead,2012,1,14,Rams,Michael Brockers,DT,21.0,2019.0,0,...,23.0,LSU,77.0,322.0,5.31,26.5,NaN,105.0,7.46,4.81
1,Les Snead,2012,2,33,Rams,Brian Quick,WR,23.0,2018.0,0,...,0.0,Appalachian State,75.0,220.0,4.50,34.0,15.0,119.0,7.10,4.23
2,Les Snead,2012,2,39,Rams,Janoris Jenkins,DB,23.0,2019.0,0,...,2.0,North Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Les Snead,2012,2,50,Rams,Isaiah Pead,RB,22.0,2016.0,0,...,0.0,Cincinnati,70.0,197.0,4.39,33.0,NaN,116.0,6.95,4.32
4,Les Snead,2012,3,65,Rams,Trumaine Johnson,DB,22.0,2019.0,0,...,0.0,Montana,74.0,204.0,4.50,35.5,19.0,122.0,7.20,4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,Ryan Pace,2019,3,73,Bears,David Montgomery,RB,22.0,2019.0,0,...,0.0,Iowa State,70.0,222.0,4.63,28.5,15.0,121.0,NaN,NaN
1594,Ryan Pace,2019,4,126,Bears,Riley Ridley,WR,23.0,2019.0,0,...,0.0,Georgia,73.0,199.0,4.58,30.5,13.0,124.0,7.22,4.28
1595,Ryan Pace,2019,6,205,Bears,Duke Shelley,CB,22.0,2019.0,0,...,0.0,Kansas State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,Ryan Pace,2019,7,222,Bears,Kerrith Whyte Jr,RB,22.0,2019.0,0,...,0.0,Florida Atlantic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
full_draft_df = processing.combine_and_clean_all_drafts()
curr_execs_df = processing.get_current_executives_and_clean()

In [75]:
df = full_draft_df.merge(curr_execs_df, how = 'inner', left_on = 'Tm', right_on = 'Teams')

In [77]:
df = df[[df.loc[x]['YEAR'] in df.loc[x]['Years Worked'] for x in range(len(df))]]

In [79]:
df.drop(['Years Worked', 'From', 'To_y', 'Teams', 'Titles'], axis = 1, inplace = True, errors = 'ignore')
df.rename({'Person' : 'GM'}, axis = 1, inplace = True)
df.rename({'To_x' : 'Played To'}, axis = 1, inplace = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [84]:
df = df[['GM', 'YEAR', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'Played To', 
            'AP1', 'PB', 'St', 'G', 'Passing Cmp', 'Passing Att', 'Passing Yds', 'Passing TD',
            'Passing Int', 'Rushing Att', 'Rushing Yds', 'Rushing TD', 'Rec',
            'Receiving Yds', 'Receiving TD', 'Solo', 'Int', 'Sk', 'College/Univ',
            'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone',
            'Shuttle']].reset_index(drop = True)

In [85]:
df

,GM,YEAR,Rnd,Pick,Tm,Player,Pos,Age,Played To,AP1,...,Sk,College/Univ,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,Les Snead,2012,1,14,Rams,Michael Brockers,DT,21.0,2019.0,0,...,23.0,LSU,77.0,322.0,5.31,26.5,NaN,105.0,7.46,4.81
1,Les Snead,2012,2,33,Rams,Brian Quick,WR,23.0,2018.0,0,...,0.0,Appalachian State,75.0,220.0,4.50,34.0,15.0,119.0,7.10,4.23
2,Les Snead,2012,2,39,Rams,Janoris Jenkins,DB,23.0,2019.0,0,...,2.0,North Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Les Snead,2012,2,50,Rams,Isaiah Pead,RB,22.0,2016.0,0,...,0.0,Cincinnati,70.0,197.0,4.39,33.0,NaN,116.0,6.95,4.32
4,Les Snead,2012,3,65,Rams,Trumaine Johnson,DB,22.0,2019.0,0,...,0.0,Montana,74.0,204.0,4.50,35.5,19.0,122.0,7.20,4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,Ryan Pace,2019,3,73,Bears,David Montgomery,RB,22.0,2019.0,0,...,0.0,Iowa State,70.0,222.0,4.63,28.5,15.0,121.0,NaN,NaN
1594,Ryan Pace,2019,4,126,Bears,Riley Ridley,WR,23.0,2019.0,0,...,0.0,Georgia,73.0,199.0,4.58,30.5,13.0,124.0,7.22,4.28
1595,Ryan Pace,2019,6,205,Bears,Duke Shelley,CB,22.0,2019.0,0,...,0.0,Kansas State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,Ryan Pace,2019,7,222,Bears,Kerrith Whyte Jr,RB,22.0,2019.0,0,...,0.0,Florida Atlantic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
round_picks_by_gm = analysis_df.groupby(['GM', 'Rnd'])['Pos'].value_counts().unstack().fillna(0)

In [164]:
round_picks_by_gm.loc['Dave Gettleman'].loc[1]

Pos
C      0.0
CB     1.0
DB     0.0
DE     0.0
DL     0.0
DT     3.0
FB     0.0
G      0.0
ILB    0.0
K      0.0
LB     0.0
LS     0.0
NT     0.0
OL     0.0
OLB    1.0
P      0.0
QB     1.0
RB     2.0
S      0.0
T      0.0
TE     0.0
WR     1.0
Name: 1, dtype: float64

In [165]:
round_picks = analysis_df.groupby('Rnd')['Pos'].value_counts().unstack().fillna(0)

In [166]:
round_picks

Pos,C,CB,DB,DE,DL,DT,FB,G,ILB,K,...,NT,OL,OLB,P,QB,RB,S,T,TE,WR
Rnd,,,,,,,,,,,,,,,,,,,,,
1,5.0,14.0,13.0,21.0,0.0,22.0,0.0,9.0,4.0,0.0,...,0.0,0.0,9.0,0.0,17.0,11.0,8.0,25.0,6.0,20.0
2,7.0,25.0,12.0,17.0,0.0,13.0,0.0,9.0,2.0,1.0,...,1.0,0.0,10.0,0.0,6.0,17.0,10.0,16.0,15.0,27.0
3,5.0,20.0,17.0,22.0,0.0,21.0,0.0,13.0,4.0,0.0,...,0.0,5.0,7.0,0.0,9.0,21.0,10.0,13.0,11.0,31.0
4,7.0,12.0,18.0,21.0,0.0,14.0,0.0,19.0,6.0,1.0,...,1.0,3.0,5.0,1.0,10.0,22.0,16.0,8.0,13.0,28.0
5,3.0,19.0,27.0,13.0,0.0,14.0,0.0,14.0,2.0,4.0,...,1.0,3.0,7.0,5.0,7.0,20.0,9.0,13.0,14.0,26.0
6,8.0,19.0,16.0,15.0,0.0,17.0,7.0,12.0,5.0,3.0,...,1.0,8.0,5.0,2.0,10.0,26.0,15.0,14.0,8.0,36.0
7,6.0,12.0,27.0,27.0,4.0,23.0,2.0,7.0,4.0,5.0,...,1.0,5.0,11.0,2.0,11.0,26.0,6.0,15.0,19.0,28.0


In [172]:
curr_execs

,Person,Teams,From,To,Titles,Years Worked
0,Brian Gutekunst,Packers,2018,2019,General Manager,"[2018, 2019]"
1,Tom Telesco,Chargers,2013,2019,General Manager,"[2013, 2014, 2015, 2016, 2017, 2018, 2019]"
2,Mike Mayock,Raiders,2019,2019,General Manager,[2019]
3,John Elway,Broncos,2011,2019,"Director of Player Personnel, Executive VP of ...","[2011, 2012, 2013, 2014, 2015, 2016, 2017, 201..."
4,John Dorsey,Chiefs,2013,2016,General Manager,"[2013, 2014, 2015, 2016]"
5,Brett Veach,Chiefs,2017,2019,General Manager,"[2017, 2018, 2019]"
6,Mike Brown,Bengals,1991,2019,Principal Owner/President/General Manager,"[1991, 1992, 1993, 1994, 1995, 1996, 1997, 199..."
7,Bill Belichick,Browns,1992,1995,Head Coach/de facto General Manager,"[1992, 1993, 1994, 1995]"
8,John Dorsey,Browns,2017,2019,General Manager,"[2017, 2018, 2019]"
9,Kevin Colbert,Steelers,2000,2019,"General Manager, VP & General Manager","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."


In [170]:
analysis_df['Pos'].value_counts()

WR     196
RB     143
LB     138
DE     136
DB     130
DT     124
CB     121
T      104
TE      86
G       83
S       74
QB      70
OLB     54
C       41
ILB     27
OL      24
K       14
P       10
FB       9
LS       5
NT       5
DL       4
Name: Pos, dtype: int64

In [171]:
analysis_df[analysis_df['Pos'] == 'DB']

,Rnd,Pick,Tm,Player,Pos,Age,Played To,AP1,PB,St,...,YEAR,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,GM
21,2,39,Rams,Janoris Jenkins,DB,23.0,2019.0,0,1,8,...,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Les Snead
23,3,65,Rams,Trumaine Johnson,DB,22.0,2019.0,0,0,5,...,2012,74.0,204.0,4.50,35.5,19.0,122.0,7.20,4.15,Les Snead
31,3,71,Rams,T.J. McDonald,DB,22.0,2018.0,0,0,6,...,2013,74.0,219.0,4.59,40.0,19.0,131.0,6.89,4.20,Les Snead
34,5,149,Rams,Brandon McGee,DB,22.0,2015.0,0,0,0,...,2013,71.0,193.0,4.40,33.5,14.0,119.0,6.71,4.18,Les Snead
38,2,41,Rams,Lamarcus Joyner,DB,23.0,2019.0,0,0,3,...,2014,68.0,184.0,4.55,37.5,14.0,124.0,7.26,4.40,Les Snead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,7,216,Panthers,D.J. Campbell,DB,23.0,2013.0,0,0,0,...,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marty Hurney
2713,4,128,Panthers,Tre Boston,DB,22.0,2019.0,0,0,4,...,2014,72.0,204.0,4.58,35.0,18.0,116.0,7.04,4.31,Marty Hurney
2714,4,128,Panthers,Tre Boston,DB,22.0,2019.0,0,0,4,...,2014,72.0,204.0,4.58,35.0,18.0,116.0,7.04,4.31,Dave Gettleman
2715,5,148,Panthers,Bene Benwikere,DB,23.0,2018.0,0,0,1,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marty Hurney


In [90]:
processing.combine_drafts_and_executives()

,GM,YEAR,Rnd,Pick,Tm,Player,Pos,Age,Played To,AP1,...,Sk,College/Univ,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
0,Les Snead,2012,1,14,Rams,Michael Brockers,DT,21.0,2019.0,0,...,23.0,LSU,77.0,322.0,5.31,26.5,NaN,105.0,7.46,4.81
1,Les Snead,2012,2,33,Rams,Brian Quick,WR,23.0,2018.0,0,...,0.0,Appalachian State,75.0,220.0,4.50,34.0,15.0,119.0,7.10,4.23
2,Les Snead,2012,2,39,Rams,Janoris Jenkins,DB,23.0,2019.0,0,...,2.0,North Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Les Snead,2012,2,50,Rams,Isaiah Pead,RB,22.0,2016.0,0,...,0.0,Cincinnati,70.0,197.0,4.39,33.0,NaN,116.0,6.95,4.32
4,Les Snead,2012,3,65,Rams,Trumaine Johnson,DB,22.0,2019.0,0,...,0.0,Montana,74.0,204.0,4.50,35.5,19.0,122.0,7.20,4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,Ryan Pace,2020,5,155,Bears,Trevis Gipson,DE,NaN,NaN,0,...,0.0,Tulsa,75.0,261.0,NaN,NaN,25.0,NaN,NaN,NaN
1851,Ryan Pace,2020,5,163,Bears,Kindle Vildor,CB,NaN,NaN,0,...,0.0,Georgia Southern,70.0,191.0,4.44,39.5,22.0,133.0,7.14,4.28
1852,Ryan Pace,2020,5,173,Bears,Darnell Mooney,WR,NaN,NaN,0,...,0.0,Tulane,70.0,176.0,4.38,37.0,9.0,124.0,NaN,NaN
1853,Ryan Pace,2020,7,226,Bears,Arlington Hambright,OT,NaN,NaN,0,...,0.0,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Need to map DL : DE